# Protein embeddings on the cloud (Google Colab, Kaggle)

### 0 - Installation and libraries

In [2]:
!pip install bio_embeddings==0.2.2

     |████████████████████████████████| 105 kB 528 kB/s eta 0:00:01
     |████████████████████████████████| 22.3 MB 29.4 MB/s eta 0:00:01
     |████████████████████████████████| 109 kB 62.3 MB/s eta 0:00:01
     |████████████████████████████████| 24.2 MB 481 kB/s  eta 0:00:01
     |████████████████████████████████| 881.9 MB 13 kB/s s eta 0:00:01     |██████████                      | 272.8 MB 2.4 MB/s eta 0:04:13     |██████████                      | 274.6 MB 2.4 MB/s eta 0:04:12     |██████████                      | 277.1 MB 2.4 MB/s eta 0:04:11     |██████████                      | 278.6 MB 2.4 MB/s eta 0:04:10     |██████████▍                     | 286.6 MB 2.4 MB/s eta 0:04:07     |███████████                     | 303.6 MB 6.8 MB/s eta 0:01:26     |███████████                     | 305.6 MB 6.8 MB/s eta 0:01:25     |███████████████▋                | 431.4 MB 16.3 MB/s eta 0:00:28     |██████████████████████▉         | 628.3 MB 26.8 MB/s eta 0:00:10     |██████████████████████▉ 

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from Bio import SeqIO
from datetime import date
from bio_embeddings.embed import ProtTransBertBFDEmbedder

### 1 - Compute protein embeddings

In [ ]:
def protein_embeddings(rbp_file_names, data_dir, results_dir):
    """
    This functions computes ProttransBERT embeddings for phage RBPs. It uses the bio-embeddings
    package together with BioPython to transform fasta sequences of phage RBPs into numerical embeddings.
    
    WARNING: running this on a personal laptop without GPU is very slow and not advised. Best to start a Google
    Collab notebook and run the computations in the cloud 
    
    Input:
    - rbp_file_names: list of fasta filenames of phage RBPs
    - data_dir: location of the sequence file(s) to loop over
    - results_dir: directory to store the resulting embeddings in
    
    Output:
    - embeddings: Pandas DataFrame (as .csv file) with file names and their corresponding embedding
    """
    embedder = ProtTransBertBFDEmbedder()
    embeddings_list = []

    for name in tqdm(rbp_file_names):
        for record in SeqIO.parse(data_dir+'/'+name, 'fasta'):
            rbp_sequence = str(record.seq)
        
        embedding = embedder.embed(rbp_sequence)
        reduced_embedding = embedder.reduce_per_protein(embedding)
        embeddings_list.append(reduced_embedding)
        
    embeddings_df = pd.concat([pd.DataFrame(rbp_file_names), pd.DataFrame(embeddings_list)], axis=1)
    embeddings_df.to_csv(results_dir+'/protein_embeddings.csv', index=False)
    
    return

In [11]:
def protein_embeddings_df(dataframe, results_dir, prefix='annotated_RBPs'):
    """
    This functions computes ProttransBERT embeddings protein sequences in a dataframe. It uses the bio-embeddings
    package together with BioPython to transform fasta sequences of phage RBPs into numerical embeddings.
    
    WARNING: running this on a personal laptop without GPU is very slow and not advised. Best to start a Google
    Collab notebook and run the computations in the cloud 
    
    Input:
    - dataframe: with protein sequences under ProteinSeq and columns NCBI_id, UniProt_id
    - results_dir: directory to store the resulting embeddings in
    
    Output:
    - embeddings: Pandas DataFrame (as .csv file) with file names and their corresponding embedding
    """
    embedder = ProtTransBertBFDEmbedder()
    embeddings_list = []
    pb = tqdm(total=dataframe.shape[0])

    for i, sequence in enumerate(dataframe['ProteinSeq']):
        embedding = embedder.embed(sequence)
        reduced_embedding = embedder.reduce_per_protein(embedding)
        embeddings_list.append(reduced_embedding)
        pb.update(1)
    pb.close()
    embeddings_df = pd.concat([dataframe['NCBI_id'].reset_index(drop=True), dataframe['UniProt_id'].reset_index(drop=True),
                               pd.DataFrame(embeddings_list)], axis=1)
    embeddings_df.to_csv(results_dir+prefix+'_embeddings.csv', index=False)
    
    return embeddings_list

In [5]:
rbps = pd.read_csv('../input/protein-sequences/annotated_RBPs.csv', sep='\t')
nonrbps = pd.read_csv('../input/protein-sequences/annotated_nonRBPs.csv', sep='\t')

In [39]:
embeddings_df.to_csv('./annotated_nonRBPs_embeddings.csv', index=False)

In [12]:
# compute RBP embeddings
protein_embeddings_df(rbps, './')

# make nonRBP subset
nonrbps_sub = nonrbps.sample(n=rbps.shape[0]*10, axis=0)

# compute nonRBP embeddings
emb_list = protein_embeddings_df(nonrbps_sub, './', prefix='annotated_nonRBPs')

Some weights of the model checkpoint at /root/.cache/bio_embeddings/prottrans_bert_bfd/model_directory were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 46910/46910 [55:05<00:00, 14.19it/s]  
